In [5]:
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark = SparkSession.builder.appName("CovidDataETL").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
data_raw = spark.read.option("multiline", "true").json("s3a://bucket-rawdata-p3-telematica/covid_data.json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Mostrar los primeros registros para confirmar la estructura inferida
data_raw.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Eliminar filas que estén completamente vacías
data_cleaned = data_raw.na.drop(how='all')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Aplicar otros filtros necesarios, ej, casos confirmados si 'estado' existe
if 'estado' in data_cleaned.columns:
    data_filtered = data_cleaned.filter(data_cleaned['estado'].isin(['Leve', 'Moderado', 'Grave']))
else:
    data_filtered = data_cleaned

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Cachear los datos limpios antes de escribir
data_filtered.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[_corrupt_record: string]

In [ ]:
# Guardar los datos procesados en S3 zona Trusted
data_filtered.write.mode('overwrite').parquet("s3a://bucket-trusteddata-p3-telematica/cleaned_covid_data")

In [ ]:
# Leer los datos procesados desde el bucket Trusted
data_trusted = spark.read.parquet("s3a://bucket-trusteddata-p3-telematica/cleaned_covid_data")

In [ ]:
# Guardar los datos procesados en S3 zona Refined en formato CSV
data_trusted.write.mode('overwrite').option("header", "true").csv("s3a://bucket-refineddata-p3-telematica/refined_covid_data_csv")

In [ ]:
# Leer los datos procesados desde el bucket Refined para confirmar
data_refined = spark.read.option("header", "true").csv("s3a://bucket-refineddata-p3-telematica/refined_covid_data_csv")

In [ ]:
# Mostrar una muestra de los datos procesados para confirmar que están bien
data_refined.show(10)

In [ ]:
# Contar la cantidad de registros en cada etapa
print(f"Cantidad de registros sin filtrar: {data_raw.count()}")
print(f"Cantidad de registros después de eliminar filas completamente nulas: {data_cleaned.count()}")
print(f"Cantidad de registros en el CSV refinado: {data_refined.count()}")